In [ ]:
!pip install langgraph

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

def run_workflow_approach():
    print("\n" + "="*25 + " WORKFLOW APPROACH " + "="*25)
    workflow = StateGraph(dict)

    def step1_extract(state):
        print("Step 1: Extract skills (always happens)")
        return {"skills": ["Python", "Java"]}

    def step2_score(state):
        print("Step 2: Score candidate (always happens)")
        return {"score": len(state.get("skills", [])) * 20}

    def step3_decide(state):
        print("Step 3: Make decision (always happens)")
        decision = "Interview" if state.get("score", 0) >= 40 else "Reject"
        return {"decision": decision}

    workflow.add_node("extract", step1_extract)
    workflow.add_node("score", step2_score)
    workflow.add_node("decide", step3_decide)
    workflow.set_entry_point("extract")
    workflow.add_edge("extract", "score")
    workflow.add_edge("score", "decide")
    workflow.add_edge("decide", END)

    app = workflow.compile()
    result = app.invoke({})
    print(f"Final Decision: {result['decision']}")

def run_agent_approach():
    print("\n" + "="*26 + " AGENT APPROACH " + "="*27)
    agent_workflow = StateGraph(dict)

    def agent_analyze(state):
        resume = state["resume"]
        print(f"Agent thinking about: '{resume}'")
        if "github" in resume.lower():
            action = "verify_github"
        elif "senior" in resume.lower():
            action = "deep_technical_check"
        else:
            action = "basic_screening"
        print(f"-> Agent decided: {action}")
        return {"action": action}

    def verify_github(state):
        print("-> Agent Action: Checking GitHub profile...")
        return {"score": 90, "notes": "GitHub verified, high confidence."}

    def deep_technical_check(state):
        print("-> Agent Action: Running deep technical assessment...")
        return {"score": 85, "notes": "Deep technical skills confirmed."}

    def basic_screening(state):
        print("-> Agent Action: Basic screening...")
        return {"score": 60, "notes": "Standard process."}

    def agent_decide(state):
        decision = "Fast-track interview" if state.get("score", 0) > 80 else "Standard interview"
        return {"decision": decision}

    agent_workflow.add_node("analyze", agent_analyze)
    agent_workflow.add_node("verify_github", verify_github)
    agent_workflow.add_node("deep_technical_check", deep_technical_check)
    agent_workflow.add_node("basic_screening", basic_screening)
    agent_workflow.add_node("decide", agent_decide)

    agent_workflow.set_entry_point("analyze")
    agent_workflow.add_conditional_edges(
        "analyze",
        lambda state: state["action"],
        {
            "verify_github": "verify_github",
            "deep_technical_check": "deep_technical_check",
            "basic_screening": "basic_screening"
        }
    )
    agent_workflow.add_edge("verify_github", "decide")
    agent_workflow.add_edge("deep_technical_check", "decide")
    agent_workflow.add_edge("basic_screening", "decide")
    agent_workflow.add_edge("decide", END)

    app = agent_workflow.compile()

    test_cases = [
        "Python developer with GitHub: github.com/johndoe",
        "Senior Python architect with 10 years experience",
        "Junior developer looking for first job"
    ]

    for resume in test_cases:
        print(f"\n--- Processing Resume: '{resume}' ---")
        result = app.invoke({"resume": resume})
        print(f"Final Decision: {result['decision']}")

if __name__ == "__main__":
    run_workflow_approach()
    run_agent_approach()


========================= WORKFLOW APPROACH =========================
Step 1: Extract skills (always happens)
Step 2: Score candidate (always happens)
Step 3: Make decision (always happens)
Final Decision: Interview

========================== AGENT APPROACH ===========================

--- Processing Resume: 'Python developer with GitHub: github.com/johndoe' ---
Agent thinking about: 'Python developer with GitHub: github.com/johndoe'
-> Agent decided: verify_github
-> Agent Action: Checking GitHub profile...
Final Decision: Fast-track interview

--- Processing Resume: 'Senior Python architect with 10 years experience' ---
Agent thinking about: 'Senior Python architect with 10 years experience'
-> Agent decided: deep_technical_check
-> Agent Action: Running deep technical assessment...
Final Decision: Fast-track interview

--- Processing Resume: 'Junior developer looking for first job' ---
Agent thinking about: 'Junior developer looking for first job'
-> Agent decided: basic_screening